In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install scikit-surprise


In [1]:
import pandas as pd
import numpy as np
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise.accuracy import rmse
from sklearn.metrics import mean_squared_error

In [2]:
train_df = pd.read_csv('/kaggle/input/alx-movie-recommendation-project-2024/train.csv')
test_df = pd.read_csv('/kaggle/input/alx-movie-recommendation-project-2024/test.csv')

In [3]:
train_df.head()

,userId,movieId,rating,timestamp
0,5163,57669,4.0,1518349992
1,106343,5,4.5,1206238739
2,146790,5459,5.0,1076215539
3,106362,32296,2.0,1423042565
4,9041,366,3.0,833375837


In [4]:
reader = Reader(rating_scale = (1,5))
data = Dataset.load_from_df(train_df[['userId','movieId','rating']],reader)
train_df_set,test_df_set = train_test_split(data,test_size=0.2)


In [7]:
model = SVD()
model.fit(train_df_set)

In [11]:
# testing our model
predict = model.test(test_df_set)
rmse_val = rmse(predict)
print(rmse_val)

RMSE: 0.8340
0.8339897676605594


In [10]:
test_df.head()

,userId,movieId
0,1,2011
1,1,4144
2,1,5767
3,1,6711
4,1,7318


In [17]:
def svd_predictor(userId,movieId):
    return model.predict(userId,movieId).est

In [21]:
test_df['rating']= test_df.apply(lambda row:svd_predictor(row['userId'],row['movieId']),axis=1)
                           

In [22]:
test_df.head()

,userId,movieId,rating
0,1,2011,3.107114
1,1,4144,3.859962
2,1,5767,3.766067
3,1,6711,3.975741
4,1,7318,3.097646


In [24]:
test_df['id']= test_df['userId'].astype(str)+'_'+ test_df['movieId'].astype(str)

In [25]:
test_df.head()

,userId,movieId,rating,id
0,1,2011,3.107114,1_2011
1,1,4144,3.859962,1_4144
2,1,5767,3.766067,1_5767
3,1,6711,3.975741,1_6711
4,1,7318,3.097646,1_7318


In [29]:
submission = test_df[['id','rating']]

In [30]:
submission.head()

,id,rating
0,1_2011,3.107114
1,1_4144,3.859962
2,1_5767,3.766067
3,1_6711,3.975741
4,1_7318,3.097646


In [31]:
submission.shape

(5000019, 2)

In [33]:
submission.to_csv('submission7',index=False)